Segmentation

In [27]:
import os
import pandas as pd
import numpy as np
import cv2
import pydicom as pdcm
from tqdm import tqdm

def preprocess(img, mask):
    if mask.shape != img.shape:
        mask=resize_mask(mask, img.shape)
    rows_to_keep = np.any(img != 0, axis=1)
    cols_to_keep = np.any(img != 0, axis=0)
    img = img[rows_to_keep][:, cols_to_keep]
    mask = mask[rows_to_keep][:, cols_to_keep]
    return img, mask

def find_bboxes_from_mask(mask, padding=10):
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    bboxes = []

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        x1 = max(0, x - padding)
        y1 = max(0, y - padding)
        x2 = min(mask.shape[1], x + w + padding)
        y2 = min(mask.shape[0], y + h + padding)
        
        # 检查是否与已有的框重叠
        overlap = False
        for (bx1, by1, bx2, by2) in bboxes:
            if x1 >= bx1 and y1 >= by1 and x2 <= bx2 and y2 <= by2:
                overlap = True
                break
        
        if not overlap:
            bboxes.append((x1, y1, x2, y2))
    return bboxes
def resize_mask(base_mask, target_shape):
    resized_mask = np.zeros(target_shape, dtype=np.uint8)
    min_rows = min(base_mask.shape[0], target_shape[0])
    min_cols = min(base_mask.shape[1], target_shape[1])
    resized_mask[:min_rows, :min_cols] = base_mask[:min_rows, :min_cols]
    return resized_mask
# 读取CSV文件并更新image_path列
dataset_dir = "/Volumes/图图/CBIS-DDSM_kaggle"
df = pd.read_csv(f'{dataset_dir}/csv/dicom_info.csv')
df['image_path'] = df.image_path.apply(lambda x: x.replace('CBIS-DDSM', dataset_dir))
df = df[df['PatientID'].str.contains('mass', case=False)]
# 初始化字典保存数据
masks = {}

# 遍历DataFrame的每一行
for index, row in tqdm(df.iterrows()):
    series_desc = row['SeriesDescription']
    patient_id = row['PatientID']
    
    if series_desc == 'full mammogram images':
        img_path = row['image_path']
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if patient_id in masks:
            if masks[patient_id].shape != img.shape:
                masks[patient_id] = resize_mask(masks[patient_id], img.shape)
        else:
            mask = np.zeros_like(img, dtype=np.uint8)
            masks[patient_id] = mask
    elif series_desc == 'ROI mask images':
        base_patient_id = '_'.join(patient_id.split('_')[:-1])
        mask_path = row['image_path']
        roi_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if base_patient_id not in masks:
            masks[base_patient_id] = roi_mask
        else:
            if masks[base_patient_id].shape != roi_mask.shape:
                masks[base_patient_id] = resize_mask(masks[base_patient_id], roi_mask.shape)
            masks[base_patient_id] = np.maximum(masks[base_patient_id], roi_mask)

# 保存图像和mask
target_dir = "/Volumes/图图/CBIS-DDSM_kaggle/process"
for patient_id, mask in masks.items():
    img_path = df.loc[df['PatientID'] == patient_id, 'image_path'].values[0]
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    img, mask = preprocess(img, mask)
    bboxes = find_bboxes_from_mask(mask)
    
    folder_type = "Train" if "Training" in patient_id else "Test"
    output_dir = os.path.join(target_dir, folder_type, patient_id)
    os.makedirs(output_dir, exist_ok=True)
    
    img_output_path = os.path.join(output_dir, 'img.jpg')
    cv2.imwrite(img_output_path, cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8))
    
    mask_output_path = os.path.join(output_dir, 'mask.png')
    cv2.imwrite(mask_output_path, mask)
    
    bbox_output_path = os.path.join(output_dir, 'bboxes.npy')
    np.save(bbox_output_path, bboxes)
    
    print(f'Saved {patient_id} images and masks to {output_dir}')


4984it [29:22,  2.83it/s]


Saved Mass-Training_P_01754_RIGHT_CC images and masks to /Volumes/图图/CBIS-DDSM_kaggle/process/Train/Mass-Training_P_01754_RIGHT_CC
Saved Mass-Training_P_01206_RIGHT_MLO images and masks to /Volumes/图图/CBIS-DDSM_kaggle/process/Train/Mass-Training_P_01206_RIGHT_MLO
Saved Mass-Training_P_00242_RIGHT_CC images and masks to /Volumes/图图/CBIS-DDSM_kaggle/process/Train/Mass-Training_P_00242_RIGHT_CC
Saved Mass-Training_P_00122_RIGHT_MLO images and masks to /Volumes/图图/CBIS-DDSM_kaggle/process/Train/Mass-Training_P_00122_RIGHT_MLO
Saved Mass-Training_P_01175_RIGHT_MLO images and masks to /Volumes/图图/CBIS-DDSM_kaggle/process/Train/Mass-Training_P_01175_RIGHT_MLO
Saved Mass-Training_P_00411_RIGHT_CC images and masks to /Volumes/图图/CBIS-DDSM_kaggle/process/Train/Mass-Training_P_00411_RIGHT_CC
Saved Mass-Training_P_00634_RIGHT_MLO images and masks to /Volumes/图图/CBIS-DDSM_kaggle/process/Train/Mass-Training_P_00634_RIGHT_MLO
Saved Mass-Test_P_00882_RIGHT_CC images and masks to /Volumes/图图/CBIS-DDSM_

Detection

In [26]:
import os
import pandas as pd
import numpy as np
import cv2
import pydicom as pdcm
from tqdm import tqdm
def find_cropped_image_position(full_image, cropped_image):
    res = cv2.matchTemplate(full_image, cropped_image, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0] + cropped_image.shape[1], top_left[1] + cropped_image.shape[0])
    return (*top_left, *bottom_right)
dataset_dir = "/Volumes/图图/CBIS-DDSM_kaggle"
df = pd.read_csv(f'{dataset_dir}/csv/dicom_info.csv')
df['image_path'] = df.image_path.apply(lambda x: x.replace('CBIS-DDSM', dataset_dir))
df = df[df['PatientID'].str.contains('mass', case=False)]
# 初始化字典保存数据
cropped = {}
data_path='/Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM'
# 遍历DataFrame的每一行
for index, row in tqdm(df.iterrows()):
        series_desc = row['SeriesDescription']
        patient_id = row['PatientID']
        
        if series_desc == 'cropped images':
            base_patient_id = '_'.join(patient_id.split('_')[:-1])
            cropped_path = row['image_path']
            cropped_image = cv2.imread(cropped_path, cv2.IMREAD_GRAYSCALE)
            if base_patient_id.split('_')[0].split('-')[1]=='Training':
                original_image=cv2.imread(os.path.join(data_path,'Train',base_patient_id,'img.jpg'), cv2.IMREAD_GRAYSCALE)
            else:
                original_image=cv2.imread(os.path.join(data_path,'Test',base_patient_id,'img.jpg'), cv2.IMREAD_GRAYSCALE)
            boxes=find_cropped_image_position(original_image,cropped_image)
            if base_patient_id not in cropped:
                cropped[base_patient_id] = [boxes]
            else:
                cropped[base_patient_id].append(boxes)
                    
        
target_dir = "/Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM"
for patient_id, boxes in cropped.items():
    
    folder_type = "Train" if "Training" in patient_id else "Test"
    output_dir = os.path.join(target_dir, folder_type, patient_id)
    os.makedirs(output_dir, exist_ok=True)
    
    bbox_output_path = os.path.join(output_dir, 'bboxes.npy')
    np.save(bbox_output_path, boxes)
    
    print(f'Saved {patient_id} bboxes to {output_dir}')

4984it [18:37,  4.46it/s]


Saved Mass-Training_P_01265_RIGHT_MLO bboxes to /Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM/Train/Mass-Training_P_01265_RIGHT_MLO
Saved Mass-Training_P_00242_RIGHT_CC bboxes to /Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM/Train/Mass-Training_P_00242_RIGHT_CC
Saved Mass-Training_P_00634_RIGHT_MLO bboxes to /Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM/Train/Mass-Training_P_00634_RIGHT_MLO
Saved Mass-Test_P_00882_RIGHT_CC bboxes to /Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM/Test/Mass-Test_P_00882_RIGHT_CC
Saved Mass-Training_P_00241_RIGHT_CC bboxes to /Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM/Train/Mass-Training_P_00241_RIGHT_CC
Saved Mass-Test_P_00145_LEFT_CC bboxes to /Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM/Test/Mass-Test_P_00145_LEFT_CC
Saved Mass-Training_P_00348_LEFT_MLO bboxes to /Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM/Train/Mass-Training_P_00348_LEFT_MLO
Saved Mass-Training_P_00259_RIGHT_MLO bboxes to /Volumes/图图/CBIS-DDSM_kaggle/CBIS-DDSM/Train/Mass-Training_P_00259_RIGHT_MLO
Saved Mass-Training_P_00248_LE

image-Classification

In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import pydicom as pdcm

def preprocess(img):
    rows_to_keep = np.any(img != 0, axis=1)
    cols_to_keep = np.any(img != 0, axis=0)
    img = img[rows_to_keep][:, cols_to_keep]
    return img

# 读取 CSV 文件并更新 image_path 列
dataset_dir = "/Volumes/图图/CBIS-DDSM_kaggle"
df = pd.read_csv(f'{dataset_dir}/csv/dicom_info.csv')
df['image_path'] = df['image_path'].apply(lambda x: x.replace('CBIS-DDSM', dataset_dir))

# 读取另一个 CSV 文件
additional_info_path = "/Volumes/图图/CBIS-DDSM_kaggle/csv/mass_case_description_train_set.csv"  # 更新为你的文件路径
# additional_info_path = "/Volumes/图图/CBIS-DDSM_kaggle/csv/mass_case_description_test_set.csv"  # 更新为你的文件路径
# additional_info_path = "/Volumes/图图/CBIS-DDSM_kaggle/csv/calc_case_description_train_set.csv"  # 更新为你的文件路径
# additional_info_path = "/Volumes/图图/CBIS-DDSM_kaggle/csv/calc_case_description_test_set.csv"  # 更新为你的文件路径
additional_df = pd.read_csv(additional_info_path)

# 初始化字典保存数据
category = {}

# 遍历 additional_df 的每一行
target_dir = "/Volumes/图图/CBIS-DDSM_kaggle/image-classification/Train"
# target_dir = "/Volumes/图图/CBIS-DDSM_kaggle/image-classification/Test"
for index, row in tqdm(additional_df.iterrows(), total=len(additional_df)):
    patient_id = row['image file path'].split('/')[0]
    # 查找对应的图像信息
    img_row = df[df['image_path'].apply(lambda x: x.split('/')[-2]) ==row['image file path'].split('/')[-2]]
    # print(img_row)
    if img_row.empty:
        print(f"No image info for patient_id: {patient_id}")
        break
        
    
    img_path = img_row['image_path'].values[0]
    # series_desc = img_row['SeriesDescription'].values[0]
    # print(img_row['SeriesDescription'])
    # if series_desc == 'full mammogram images':
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    if img is None:
        print(f"Failed to read image: {img_path}")
        break
    
    img = preprocess(img)
    
    # 构建字典
    info_dict = {
        "breast_density": str(row['breast_density']).replace(' ',''),
        "left_or_right_breast": str(row['left or right breast']).replace(' ',''),
        "image_view": str(row['image view']).replace(' ','')
        # "abnormality_type": row['abnormality type'],
        # "assessment": row['assessment'],
        # "pathology": row['pathology'],
        # "subtlety": row['subtlety']
    }
    
    # 保存图像和字典
    output_dir = os.path.join(target_dir, patient_id)
    os.makedirs(output_dir, exist_ok=True)
    
    img_output_path = os.path.join(output_dir, 'img.jpg')
    cv2.imwrite(img_output_path, cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8))
    
    dict_output_path = os.path.join(output_dir, 'info_dict.npy')
    np.save(dict_output_path, info_dict)
    
    print(f'Saved {patient_id} images and categories to {output_dir}')

print("Processing complete.")


  0%|          | 1/1318 [00:00<13:39,  1.61it/s]

Saved Mass-Training_P_00001_LEFT_CC images and categories to /Volumes/图图/CBIS-DDSM_kaggle/image-classification/Train/Mass-Training_P_00001_LEFT_CC


  0%|          | 2/1318 [00:01<16:10,  1.36it/s]

Saved Mass-Training_P_00001_LEFT_MLO images and categories to /Volumes/图图/CBIS-DDSM_kaggle/image-classification/Train/Mass-Training_P_00001_LEFT_MLO


  0%|          | 3/1318 [00:02<14:36,  1.50it/s]

...

Saved Mass-Training_P_02079_RIGHT_MLO images and categories to /Volumes/图图/CBIS-DDSM_kaggle/image-classification/Train/Mass-Training_P_02079_RIGHT_MLO


100%|█████████▉| 1317/1318 [16:52<00:01,  1.29s/it]

Saved Mass-Training_P_02092_LEFT_CC images and categories to /Volumes/图图/CBIS-DDSM_kaggle/image-classification/Train/Mass-Training_P_02092_LEFT_CC


100%|██████████| 1318/1318 [16:53<00:00,  1.30it/s]

Saved Mass-Training_P_02092_LEFT_MLO images and categories to /Volumes/图图/CBIS-DDSM_kaggle/image-classification/Train/Mass-Training_P_02092_LEFT_MLO
Processing complete.


cropped-classification

In [5]:
import os
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import pydicom as pdcm

def preprocess(img):
    rows_to_keep = np.any(img != 0, axis=1)
    cols_to_keep = np.any(img != 0, axis=0)
    img = img[rows_to_keep][:, cols_to_keep]
    return img

# 读取 CSV 文件并更新 image_path 列
dataset_dir = "/Volumes/图图/CBIS-DDSM_kaggle"
df = pd.read_csv(f'{dataset_dir}/csv/dicom_info.csv')
df['image_path'] = df['image_path'].apply(lambda x: x.replace('CBIS-DDSM', dataset_dir))

# 读取另一个 CSV 文件
additional_info_path = "/Volumes/图图/CBIS-DDSM_kaggle/csv/calc_case_description_train_set.csv"  # 更新为你的文件路径
additional_df = pd.read_csv(additional_info_path)

# 初始化字典保存数据
category = {}

# 遍历 additional_df 的每一行
target_dir = "/Volumes/图图/CBIS-DDSM_kaggle/cropped-classification/Train"
for index, row in tqdm(additional_df.iterrows(), total=len(additional_df)):
    patient_id = row['cropped image file path'].split('/')[0]
    # 查找对应的图像信息
    img_row = df[df['image_path'].apply(lambda x: x.split('/')[-2]) ==row['cropped image file path'].split('/')[-2]]
    # print(img_row)
    if img_row.empty:
        print(f"No image info for patient_id: {patient_id}")
        break
        
    try:
        img_path = img_row[img_row['SeriesDescription']=='cropped images']['image_path'].values[0]
    except:
        print(img_row)

    # series_desc = img_row['SeriesDescription'].values[0]
    # print(img_row['SeriesDescription'])
    # if series_desc == 'full mammogram images':
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    if img is None:
        print(f"Failed to read image: {img_path}")
        break
    
    # img = preprocess(img)
    
    # 构建字典
    info_dict = {
        # "breast_density": row['breast_density'],
        # "left_or_right_breast": row['left or right breast'],
        # "image_view": row['image view']
        "abnormality_type": str(row['abnormality type']).replace(' ',''),
        "assessment": str(row['assessment']).replace(' ',''),
        "pathology": str(row['pathology']).replace(' ',''),
        "subtlety": str(row['subtlety']).replace(' ','')
    }
    
    # 保存图像和字典
    output_dir = os.path.join(target_dir, patient_id)
    os.makedirs(output_dir, exist_ok=True)
    
    img_output_path = os.path.join(output_dir, 'img.jpg')
    cv2.imwrite(img_output_path, cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8))
    
    dict_output_path = os.path.join(output_dir, 'info_dict.npy')
    np.save(dict_output_path, info_dict)
    
    print(f'Saved {patient_id} images and categories to {output_dir}')

print("Processing complete.")


  0%|          | 1/1546 [00:00<23:10,  1.11it/s]

Saved Calc-Training_P_00005_RIGHT_CC_1 images and categories to /Volumes/图图/CBIS-DDSM_kaggle/cropped-classification/Train/Calc-Training_P_00005_RIGHT_CC_1


  0%|          | 2/1546 [00:01<13:54,  1.85it/s]

Saved Calc-Training_P_00005_RIGHT_MLO_1 images and categories to /Volumes/图图/CBIS-DDSM_kaggle/cropped-classification/Train/Calc-Training_P_00005_RIGHT_MLO_1


  0%|          | 3/1546 [00:01<10:26,  2.46it/s]

...

Saved Calc-Training_P_01563_LEFT_MLO_3 images and categories to /Volumes/图图/CBIS-DDSM_kaggle/cropped-classification/Train/Calc-Training_P_01563_LEFT_MLO_3


 79%|███████▊  | 1215/1546 [06:31<00:46,  7.08it/s]

Saved Calc-Training_P_01563_LEFT_MLO_4 images and categories to /Volumes/图图/CBIS-DDSM_kaggle/cropped-classification/Train/Calc-Training_P_01563_LEFT_MLO_4
Saved Calc-Training_P_01563_RIGHT_CC_1 images and categories to /Volumes/图图/CBIS-DDSM_kaggle/cropped-classification/Train/Calc-Training_P_01563_RIGHT_CC_1
Saved Calc-Training_P_01563_RIGHT_CC_2 images and categories to /Volumes/图图/CBIS-DDSM_kaggle/cropped-classification/Train/Calc-Training_P_01563_RIGHT_CC_2


 79%|███████▊  | 1216/1546 [06:31<01:46,  3.11it/s]

Saved Calc-Training_P_01563_RIGHT_MLO_1 images and categories to /Volumes/图图/CBIS-DDSM_kaggle/cropped-classification/Train/Calc-Training_P_01563_RIGHT_MLO_1
No image info for patient_id: Calc-Training_P_01563_RIGHT_MLO_2
Processing complete.
